# 2. Neural Style Transfer on your local machine

---

### Import package and load .env

In [ ]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import json
import os
%load_ext dotenv
%dotenv

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

### Add video to storage account

In [ ]:
!wget https://happypathspublic.blob.core.windows.net/videos/orangutan.mp4

In [ ]:
!azcopy \
    --source orangutan.mp4 \
    --destination https://{get_key(env_path, "STORAGE_ACCOUNT_NAME")}.blob.core.windows.net/{get_key(env_path, "STORAGE_CONTAINER_NAME")}/orangutan.mp4 \
    --dest-key {get_key(env_path, "STORAGE_ACCOUNT_KEY")} \
    --resume "."

In [ ]:
!rm orangutan.mp4

### Preprocess video 

`--frames-dir` is the directory in Azure blob that will be created to store individual frames from preprocessing the video.

In [ ]:
!python aci/preprocess.py --video orangutan.mp4 --frames-dir $input_frames_dir 

In [ ]:
!rm -rf orangutan*

### Test neural style transfer locally
- Add 3 images to queue
- Dequeue those 3 images and apply neural style transfer locally
- Check that the queue is now empty

In [ ]:
output_frames_dir = "processed_orangutan_frames_test"
style = "mosaic" # the other options are in /models

In [ ]:
!python aci/add_images_to_queue.py \
    --input-dir $input_frames_dir \
    --output-dir $output_frames_dir \
    --style $style \
    --queue-limit 3

The `--input-dir` is the directory in your blob container that corresponds to the queue messages (the script will look inside the directory specified by `--input-dir` to find and process the message that it dequeued).

In [ ]:
!python aks/process_images_from_queue.py --terminate

In [ ]:
!az servicebus queue show \
    --name {get_key(env_path, "SB_QUEUE")} \
    --namespace-name {get_key(env_path, "SB_NAMESPACE")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --query "countDetails.activeMessageCount"

---

### Conclusion

Check that our `.env` file looks correct.

In [ ]:
!cat .env